In [1]:
%%configure -f
{"driverMemory": "4G", 
 "executorMemory": "2G", 
 "executorCores": 2, 
 "numExecutors": 2,
 "conf": {"spark.dynamicAllocation.enabled":"false", "spark.sql.broadcastTimeout":2000}
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
139,application_1627924993397_0063,pyspark,idle,Link,Link,
140,application_1627924993397_0072,pyspark,shutting_down,Link,Link,


In [2]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
141,application_1627924993397_0076,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType, LongType, IntegerType, DoubleType, StructType, StructField

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [287]:
def get_top_user_categories(spark, model_predictions_path, top_k):
    transform_expr = "TRANSFORM(category_trend_scores, x -> struct(x.probability*(-1.00) as probability, x.category_id as category_id))"

    top_predictions_per_user = (spark.read.parquet(model_predictions_path)
                                .select("customer_id", F.expr(transform_expr).alias("category_transform"))
                                .withColumn("category_sorted", F.array_sort(F.col("category_transform")))
                                .withColumn("topk_rank_cat",
                                            F.array([F.col("category_sorted")[i] for i in range(top_k)]))
                                .select('customer_id',
                                        F.posexplode(F.col("topk_rank_cat")).alias("pos", "cma_id_prob"))
                                .withColumn("cma_id", F.col("cma_id_prob").getItem("category_id"))
                                .withColumn("prob", F.col("cma_id_prob").getItem("probability") * -1.00)
                                .withColumn('rank', (F.col('pos') + 1))
                                .select("rank","customer_id","cma_id","prob"))
    return top_predictions_per_user

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [288]:
def get_user_sales(spark, l1_banner_events_base_table_path):
    
    user_sales = (spark.read.parquet(l1_banner_events_base_table_path)
                .filter((F.col('sales').isNotNull()) & (F.col('sales') > 0))
                .select(F.col("customer_id").cast(LongType()), F.col("cma_id").cast(IntegerType()))
                .distinct())
    return user_sales

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [289]:
def get_sales_category_rank_per_cust (user_sales, model_predictions):

    sales_category_rank_per_cust = (user_sales
                                .join(model_predictions.select("customer_id").distinct(), on = "customer_id")
                                .join(model_predictions,on = ["customer_id","cma_id"], how = "left")
                                .fillna(999999, subset= "rank" ) # 999999 means the category is not belongs to topk rank
                                .select("customer_id","cma_id","rank"))

    return sales_category_rank_per_cust

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [290]:
def get_model_servable_category (model_serve_category_pre):
        
    model_servable_category = (model_serve_category_pre
                            .select("cma_id")
                            .distinct())

    return model_servable_category

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [291]:
def get_precision_recall_at_k_per_cust (sales_category_rank_per_cust, model_servable_category, k):
        
    count_model_serve_category = (sales_category_rank_per_cust
                                .join(model_servable_category, on = "cma_id")
                                .groupBy("customer_id")
                                .agg(F.countDistinct(F.col("cma_id")).alias("count_cat")))

    precision_per_cust_only_for_rank_les_k = (sales_category_rank_per_cust
                                            .filter(F.col("rank")<=k)
                                            .groupBy("customer_id")
                                            .agg(F.countDistinct(F.col("cma_id")).alias("count_cat_les_k_rank"))
                                            .withColumn("precision",F.when(F.col("count_cat_les_k_rank")>0,1)
                                                                    .otherwise(0)))

    precision_at_k_per_cust = (sales_category_rank_per_cust
                            .select("customer_id")
                            .distinct()
                            .join(precision_per_cust_only_for_rank_les_k, on = "customer_id", how = "left")
                            .fillna(0, subset = ["count_cat_les_k_rank","precision"])
                            .withColumn("k",F.lit(k)))

    precision_recall_at_k_per_cust = (precision_at_k_per_cust
                                    .join(count_model_serve_category, on = "customer_id")
                                    .withColumn("recall",F.col("count_cat_les_k_rank")/F.col("count_cat")))

    return precision_recall_at_k_per_cust

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [292]:
def get_precision_recall_at_k (precision_recall_at_k_per_cust):
        
    precision_recall_at_k = (precision_recall_at_k_per_cust
                            .groupBy(F.col('k'))
                            .agg(F.mean('precision').alias('precision_at_k'),
                                F.mean('recall').alias("recall_at_k"))
                            .select('k', 'precision_at_k', 'recall_at_k')
                            .orderBy(F.col('k')))

    return precision_recall_at_k

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# configs

In [359]:
top_k=6
model_name='boosting'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [442]:
model_date="2021-07-29"
target_date="2021-07-30"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# input

In [443]:
model_predictions_path = f"s3a://pai-gm-midgar-production-ml-jobs/shinra/modelling/weighted_trendfiltering_banner_events/user_categories_boosting/model_date={model_date}"

l1_banner_events_base_table_path = f"s3a://midgar-aws-workspace/prod/shinra/telescope/base/l1_banner_events_base_table/dt={target_date}"  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [444]:
model_predictions_path

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://pai-gm-midgar-production-ml-jobs/shinra/modelling/weighted_trendfiltering_banner_events/user_categories_boosting/model_date=2021-07-29'

In [445]:
l1_banner_events_base_table_path

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://midgar-aws-workspace/prod/shinra/telescope/base/l1_banner_events_base_table/dt=2021-07-30'

# Output Path


In [446]:
precisionrecallatkv_path = f"s3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/precisionrecallatkv/model={model_name}/"

precision_recall_at_k_path = f"{precisionrecallatkv_path}/dt={model_date}"
precision_recall_at_k_path


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/precisionrecallatkv/model=boosting//dt=2021-07-29'

# work

In [447]:
model_predictions = get_top_user_categories(spark, model_predictions_path, top_k).cache()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [448]:
user_sales = get_user_sales(spark, l1_banner_events_base_table_path)
        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [449]:
sales_category_rank_per_cust = get_sales_category_rank_per_cust(user_sales, model_predictions).cache()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [450]:
model_servable_category = get_model_servable_category(model_predictions).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [451]:
for k in range(1,top_k+1):
    if k == 1:
        precision_recall_at_k_per_cust = get_precision_recall_at_k_per_cust(sales_category_rank_per_cust, model_servable_category, k)
    else:
        precision_recall_at_k_per_cust_add = get_precision_recall_at_k_per_cust(sales_category_rank_per_cust, model_servable_category, k)
        precision_recall_at_k_per_cust = precision_recall_at_k_per_cust.union(precision_recall_at_k_per_cust_add)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [452]:
precision_recall_at_k = get_precision_recall_at_k(precision_recall_at_k_per_cust)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [453]:
precision_recall_at_k_path

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/precisionrecallatkv/model=boosting//dt=2021-07-29'

In [454]:
precision_recall_at_k.coalesce(1).write.mode("overwrite").parquet(precision_recall_at_k_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# validation

In [455]:
df=spark.read.parquet(precision_recall_at_k_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [456]:
df.show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+------------------+
|k  |precision_at_k    |recall_at_k       |
+---+------------------+------------------+
|1  |0.5685242622949499|0.4910470909584802|
|2  |0.7566855894276004|0.6924275461054982|
|3  |0.8459769787769785|0.7992512318924055|
|4  |0.8965903181754773|0.8623577861226651|
|5  |0.9275064154832501|0.9025763086221568|
|6  |0.9480750327906953|0.9300546664393394|
+---+------------------+------------------+

# analysis

In [4]:
base_path=f"s3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/precisionrecallatkv"
base_path

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/precisionrecallatkv'

In [5]:
df=spark.read.parquet(base_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.show(2,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+------------------+--------+----------+
|k  |precision_at_k    |recall_at_k       |model   |dt        |
+---+------------------+------------------+--------+----------+
|1  |0.6213197297507095|0.4624938343534486|dampened|2021-07-25|
|2  |0.793544457456024 |0.678032992884694 |dampened|2021-07-25|
+---+------------------+------------------+--------+----------+
only showing top 2 rows

In [22]:
df.filter(F.col('k')==6).sort(F.col('dt')).coalesce(1).write.mode("overwrite").option("header","true")\
.csv(f"s3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/precisionrecall6")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
dfa=df.groupBy('model', 'k').agg(F.mean('precision_at_k').alias('mean_precision_at_k'),
                                                      F.mean('recall_at_k').alias('mean_recall_at_k'))\
.sort(F.col('k'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
dfa.show(30,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---+-------------------+-------------------+
|model   |k  |mean_precision_at_k|mean_recall_at_k   |
+--------+---+-------------------+-------------------+
|dampened|1  |0.6131867269407321 |0.46635353475042035|
|raw     |1  |0.6128997531774739 |0.4661199835185502 |
|boosting|1  |0.6109813574809771 |0.4644059231415015 |
|prod    |1  |0.2982890577122333 |0.23560034991360662|
|raw     |2  |0.7878333162810274 |0.6788979905086405 |
|dampened|2  |0.7877724412775217 |0.678846383581239  |
|boosting|2  |0.7868471943742877 |0.6778132758988765 |
|prod    |2  |0.6048450903366591 |0.5027177199340968 |
|raw     |3  |0.8728991800159542 |0.7959962945974194 |
|dampened|3  |0.872491066224466  |0.7956004676664492 |
|prod    |3  |0.7915448151952488 |0.6934409599568471 |
|boosting|3  |0.8719509718421331 |0.794927184332761  |
|raw     |4  |0.918467717594605  |0.8649081133684238 |
|dampened|4  |0.9176245088045277 |0.8638296776299959 |
|prod    |4  |0.8776294649388305 |0.8032063275106125 |
|boosting|

In [28]:
dfa.coalesce(1).write.mode("overwrite").option("header","true").csv(f"s3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/precisionrecallagg")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…